# Adaptive PDE discretizations on Cartesian grids
## Volume : Divergence form PDEs
## Part : Linear elasticity
## Chapter : Comparisons with the Virieux and Lebedev elastic schemes

$
\newcommand\cE{\mathcal{E}}
\newcommand\<{\langle} \newcommand\>{\rangle}
\newcommand\be{\boldsymbol{e}}
\newcommand\kw{\mathrm{k}}
\newcommand\ri{\mathrm{i}}
\newcommand\rI{\mathrm{I}}
\newcommand\diff{\mathrm{d}}
\newcommand\cO{\mathcal{O}}
\DeclareMathOperator\Tr{Tr}
\newcommand\Id{\mathrm{Id}}
\newcommand\bR{\mathbb{R}}
\newcommand\bZ{\mathbb{Z}}
$

In a series of other notebooks, we introduced a discretization of the elastic pootential energy, and thus a symplectic numerical scheme for the elastic wave equation. See the notebooks on [the elastic energy](ElasticEnergy), [wave propagation](ElasticWave), [high order schemes](HighOrderWaves), [gradient backpropagation](WaveExamples). The objective of this notebook is to compare the introduces method, referred to as the symplectic method or the Selling elastic scheme, to existing methods in the literature. More precisely we consider :

- the Virieux scheme, which assumes a VTI Hooke tensor
- the Lebedev scheme, which is defined as a collection of $2^{d-1}$ coupled Virieux schemes, and allows any Hooke tensor.

The Lebedev scheme and the proposed symplectic scheme, which both allow arbitrary Hooke tensors, can be compared on the following points. 
- *energy conservation*. The symplectic scheme preserves a perturbed energy exactly, and therefore preserves the target energy up to a multiplicative constant (close to one if the CFL is satisfied). The Virieux and Lebedev schemes do not have this guarantee, and we observe strong oscillations in the energy (with random Hooke tensors).
- *grid decoupling*. The symplectic scheme preserves a coercive energy, which controls the solution regularity via the Korn inequality, and forbids e.g chessboard artifacts. The Lebedev scheme does not offer such a guarantee, and in fact the solution on two subgrids may decouple and evolve independently in a chessboard pattern.
- *numerical dispersion relation*. For a given number of discretization points per wavelength, which scheme has the most accurate numerical dispersion relation ? We observe that the Virieux scheme is the most accurate for VTI models, followed by the proposed symplectic scheme, and last the Lebedev scheme. For non-VTI models (specifically, the mica medium), the Virieux scheme is not applicable, the situation is more nuanced : the proposed symplectic scheme appears best for the pressure wave, and the Lebedev scheme for the sher wave.

We implement the Virieux and Lebedev schemes, but only in a quick and dirty way which is only meant to perform basic comparisons and check the dispersion relations.

[**Summary**](Summary.ipynb) of volume Divergence form PDEs, this series of notebooks.

[**Main summary**](../Summary.ipynb) of the Adaptive Grid Discretizations 
	book of notebooks, including the other volumes.

# Table of contents
  * [1. Staggered grids](#1.-Staggered-grids)
  * [2. The Virieux scheme](#2.-The-Virieux-scheme)
  * [3. The Lebedev scheme](#3.-The-Lebedev-scheme)
  * [4. Dispersion relations](#4.-Dispersion-relations)
  * [5. Isotropic dispersion](#5.-Isotropic-dispersion)
  * [6. Energy conservation](#6.-Energy-conservation)
  * [7. Grid decoupling in the Lebedev scheme](#7.-Grid-decoupling-in-the-Lebedev-scheme)



**Acknowledgement.** Some of the experiments presented in these notebooks are part of 
ongoing research with Ludovic Métivier and Da Chen.

Copyright Jean-Marie Mirebeau, Centre Borelli, ENS Paris-Saclay, CNRS, University Paris-Saclay

## 0. Importing the required libraries

In [3]:
import sys; sys.path.insert(0,"..") # Allow import of agd from parent directory (useless if conda package installed)
#from Miscellaneous import TocTools; print(TocTools.displayTOC('ElasticComparisons','Div'))

In [2]:
from agd.Metrics.Seismic import Hooke
from agd import FiniteDifferences as fd
from agd import LinearParallel as lp
from agd.ExportedCode.Notebooks_Div.HighOrderWaves import make_domain,dispersion_e,MakeRandomTensor
from agd.Eikonal.HFM_CUDA import AnisotropicWave

In [3]:
import numpy as np
import itertools
import copy
from matplotlib import pyplot as plt
π = np.pi
np.set_printoptions(linewidth=2000)

## 1. Staggered grids

The Virieux and Lebedev schemes rely on Staggered grids : in two dimensions, the scheme unknowns may be stored at 
\begin{align*}
    & (i,j) \diff x &
    & (i+1/2,j) \diff x &
    & (i,j+1/2) \diff x &
    & (i+1/2,j+1/2) \diff x,
\end{align*}
where $i,j \in \bZ$. This is emphasized by subscripts `_00`, `_10`, `_01`, `_11`, in the variable name. The proposed symplectic scheme, in contrast, uses the standard grid.

In [4]:
def shifted_grids(X,dx):
    """
    Shifted grids for staggered scheme.
    X_0,X_1
    X_00,X_10,X_01,X_11 (inverted binary order)
    X_000,X_100,X_010,X_110,X_001,X_101,X_011,X_111
    """
    vdim = len(X)
    shape = X[0].shape
    shifts = itertools.product((0,dx/2),repeat=len(X))
    return [X + fd.as_field(s[::-1],shape) for s in shifts]

The point of the staggered grid representation is to have second order accurate finite differences for derivatives along suitable grid axes. 
For instance
$$
    \frac{\partial}{\partial x} u( (i,j) \diff x) = \frac{u( (i+1/2,j) \diff x) - u( (i-1/2,j) \diff x)}{\diff x} + \cO(\diff x^2).
$$
Fourth order accurate finite differences likewise have a small stencil.

In [5]:
class staggered:
    """A class for first order finite difference operators on staggered grids"""
    
    def __init__(self,dx,order=2,bc='Periodic'):
        self.idx=1/dx
        self.order=order; assert order in (2,4)
        self.bc=bc; assert bc in ('Periodic','Dirichlet')

    def roll(self,q,shift,axis,bc=None):
        """Rolls specified axis, inserts zeros in case of Dirichlet boundary conditions"""
        if bc is None: bc=self.bc
        q = np.roll(q,shift,axis)
        if bc!='Periodic': 
            pad = q[*(slice(None),)*axis,shift,None] if bc=='Constant' else 0.
            if shift>0: q[*(slice(None),)*axis,:shift]=pad
            else: q[*(slice(None),)*axis,shift:]=pad
        return q
        
    def diff_left(self,q,axis,s=0): 
        """
        First order finite difference, to the left, along axis i.
        (Result is shifted by half a grid step to the left.)
        """
        dq = self.roll(q,-s,axis)-self.roll(q,1-s,axis) # Centered finite difference, with step 1/2
        if self.order==2: return dq * self.idx 
        dq2 = self.roll(q,-1-s,axis)-self.roll(q,2-s,axis) # Centered finite difference, with step 3/2
        return ( (9/8)*dq-(1/24)*dq2 )*self.idx # Fourth order finite difference
    
    def diff_right(self,q,axis):
        """
        First order finite difference, to the right, along axis i.
        (Result is shifted by half a grid step to the right.)
        """
        return self.diff_left(q,axis,s=1)

    def avg_left(self,q,axis,s=0):
        """
        Approximate value, half a grid step to the left, along axis i.
        """
        if isinstance(axis,tuple): # Average over several axes
            for ax in axis: q = self.avg_left(q,ax,s)
            return q
        bc = 'Constant'
        aq = self.roll(q,-s,axis,bc)+self.roll(q,1-s,axis,bc) # Centered average, with step 1/2
        if self.order==2: return aq * 0.5 
        aq2 = self.roll(q,-1-s,axis,bc)+self.roll(q,2-s,axis,bc) # Centered average, with step 3/2
        return (9/16)*aq - (1/16)*aq2 # Fourth order accurate interpolation 
    
    def avg_right(self,q,axis):
        return self.avg_left(q,axis,s=1)

## 2. The Virieux scheme

This scheme assumes a VTI hooke tensor. In two dimensions, the structure reads
$$
    C = \begin{pmatrix}
    * & * & \\
    * & * & \\
      &   & *
    \end{pmatrix}.
$$
In other words, $C_{02} = C_{12} = 0$. (And likewise $C_{20}=C_{21}=0$ by symmetry.)

The Virieux scheme uses a staggered grid, with the following variable locations : 
- the *position and momentum* components are stored in shifted positions along their respective axis. For instance, the unknowns associated to the position $q=(q_0,q_1)$ are for $i,j \in \bZ$
\begin{align*}
    & q_0( (i+1/2,j) \diff x), &
    & q_1( (i,j+1/2) \diff x).
\end{align*}
- *strain and stress* tensors have their diagonal elements centered, and their off-diagonal elements shifted along the diagonal. For instance, the unknowns associated to the stress tensor are for $i,j \in \bZ$
\begin{align*}
    & \sigma_{00}((i,j)\diff x), &
    & \sigma_{11}((i,j)\diff x), &
    & \sigma_{01}((i+1/2,j+1/2)\diff x).
\end{align*}

In [6]:
def eval_Virieux(qfun,pfun,X,dx,t,dt):
    """
    Evaluate position and momentum at the given position and time, 
    taking into account spatial and temporal grid shifts
    """
    vdim = len(X)
    t2 = t+dt/2
    if vdim==1:
        _,X_1 = shifted_grids(X,dx)
        q0_1 = qfun(t2,X_1)[0]
        p0_1 = pfun(t, X_1)[0]
        return (q0_1,),(p0_1,)
    if vdim==2:
        _,X_10,X_01,_ = shifted_grids(X,dx)
        q0_10 = qfun(t2,X_10)[0]
        q1_01 = qfun(t2,X_01)[1]
        p0_10 = pfun(t, X_10)[0]
        p1_01 = pfun(t, X_01)[1]
        return (q0_10,q1_01),(p0_10,p1_01)
    if vdim==3:
        X_000,X_100,X_010,X_110,X_001,X_101,X_011,X_111 = shifted_grids(X,dx)
        q0_100 = qfun(t2,X_100)[0]
        q1_010 = qfun(t2,X_010)[1]
        q2_001 = qfun(t2,X_001)[2]
        p0_100 = pfun(t, X_100)[0]
        p1_010 = pfun(t, X_010)[1]
        p2_001 = pfun(t, X_001)[2]
        return (q0_100,q1_010,q2_001),(p0_100,p1_010,p2_001)

In [7]:
class Virieux2:
    """The two dimensional Virieux scheme."""
    def __init__(self,ρ,C,stag):
        """
        Inputs : 
        - ρ : density, everywhere positive. 
        - C : Hooke tensor, assumes a VTI structure. Assumes C02=C12=0 and C symmetric.
        - stag : staggered grid difference operators
        """
        self.stag = stag
        ar = stag.avg_right
        # Variable locations on the grid shown after subscript
        iρ = 1/ρ
        self.iρ_10 = ar(iρ,0)
        self.iρ_01 = ar(iρ,1) 
        self.C00_00 = C[0,0]
        self.C01_00 = C[0,1]
        self.C11_00 = C[1,1] 
        self.C22_11 = ar(C[2,2],axis=(0,1))

    def step(self,q,p,dt):
        """
        Inputs : 
        - q,p : position and momentum.
        - dt : time step.
        """
        # Variable locations shown after subscript
        dl,dr = self.stag.diff_left, self.stag.diff_right
        C00_00,C01_00,C10_00,C11_00,C22_11,iρ_10,iρ_01 = self.C00_00,self.C01_00,self.C01_00,self.C11_00,self.C22_11,self.iρ_10,self.iρ_01

        q0_10,q1_01 = copy.deepcopy(q)
        p0_10,p1_01 = copy.deepcopy(p)
        
        # Compute the strain tensor ϵ = (Dq+Dq^T)/2
        ϵ00_00 = dl(q0_10,0) 
        ϵ11_00 = dl(q1_01,1) 
        ϵ01_11 = dr(q0_10,1) + dr(q1_01,0) # We omit the factor two in ϵ01 in view of Voigt's notation

        # Compute the stress tensor
        σ00_00 = C00_00*ϵ00_00+C01_00*ϵ11_00 
        σ11_00 = C10_00*ϵ00_00+C11_00*ϵ11_00 
        σ01_11 = C22_11*ϵ01_11 

        # Stress divergence
        dp0_10 = dr(σ00_00,0) + dl(σ01_11,1) 
        dp1_01 = dl(σ01_11,0) + dr(σ11_00,1)
#        self.tmp = (dp0_10,dp1_01),(σ00_00,σ11_00,σ01_11),(ϵ00_00,ϵ11_00,ϵ01_11)

        # Mimick symplectic updates by updating first p, and then q
        p0_10 += dt*dp0_10 
        p1_01 += dt*dp1_01 

        q0_10 += dt*p0_10*iρ_10 
        q1_01 += dt*p1_01*iρ_01

        return (q0_10,q1_01),(p0_10,p1_01)

In [8]:
class Virieux3:
    """The three dimensional Virieux scheme."""
    def __init__(self,ρ,C,stag):
        """
        Inputs : 
        - ρ : density, everywhere positive. 
        - C : Hooke tensor, assumes a VTI structure. Assumes C02=C12=0 and C symmetric.
        - stag : staggered grid difference operators
        """
        self.stag = stag
        ar = stag.avg_right
        # Variable locations on the grid shown after subscript
        iρ = 1/ρ
        self.iρ_100 = ar(iρ,0)
        self.iρ_010 = ar(iρ,1) 
        self.iρ_001 = ar(iρ,2) 
        self.C00_000 = C[0,0]
        self.C01_000 = C[0,1]
        self.C02_000 = C[0,2]
        self.C11_000 = C[1,1] 
        self.C12_000 = C[1,2] 
        self.C22_000 = C[2,2] 
        # Voigt : 3 -> (1,2), 4->(0,2), 5->(0,1)
        self.C33_011 = ar(C[3,3],axis=(1,2))
        self.C44_101 = ar(C[4,4],axis=(0,2))
        self.C55_110 = ar(C[5,5],axis=(0,1))

    def step(self,q,p,dt):
        """
        Inputs : 
        - q,p : position and momentum.
        - dt : time step.
        """
        # Variable locations shown after subscript
        dl,dr = self.stag.diff_left, self.stag.diff_right
        iρ_100,iρ_010,iρ_001, C00_000,C01_000,C02_000,C11_000,C12_000,C22_000, C33_011,C44_101,C55_110 = \
        self.iρ_100,self.iρ_010,self.iρ_001, self.C00_000,self.C01_000,self.C02_000,self.C11_000,self.C12_000,self.C22_000, self.C33_011,self.C44_101,self.C55_110

        q0_100,q1_010,q2_001 = copy.deepcopy(q)
        p0_100,p1_010,p2_001 = copy.deepcopy(p)
        
        # Compute the strain tensor ϵ = (Dq+Dq^T)/2
        ϵ00_000 = dl(q0_100,0) 
        ϵ11_000 = dl(q1_010,1)
        ϵ22_000 = dl(q2_001,2)
        ϵ01_110 = dr(q0_100,1) + dr(q1_010,0) # We omit the factor two in ϵ01 in view of Voigt's notation
        ϵ02_101 = dr(q0_100,2) + dr(q2_001,0) 
        ϵ12_011 = dr(q1_010,2) + dr(q2_001,1) 

        # Compute the stress tensor
        σ00_000 = C00_000*ϵ00_000 + C01_000*ϵ11_000 + C02_000*ϵ22_000
        σ11_000 = C01_000*ϵ00_000 + C11_000*ϵ11_000 + C12_000*ϵ22_000
        σ22_000 = C02_000*ϵ00_000 + C12_000*ϵ11_000 + C22_000*ϵ22_000
        # Voigt : 3 -> (1,2), 4->(0,2), 5->(0,1)
        σ12_011 = C33_011*ϵ12_011
        σ02_101 = C44_101*ϵ02_101
        σ01_110 = C55_110*ϵ01_110

        # Stress divergence
        dp0_100 = dr(σ00_000,0) + dl(σ01_110,1) + dl(σ02_101,2) 
        dp1_010 = dl(σ01_110,0) + dr(σ11_000,1) + dl(σ12_011,2) 
        dp2_001 = dl(σ02_101,0) + dl(σ12_011,1) + dr(σ22_000,2) 
        self.tmp = (dp0_100,dp1_010,dp2_001),(σ00_000,σ11_000,σ22_000,σ01_110,σ02_101,σ12_011),(ϵ00_000,ϵ11_000,ϵ22_000,ϵ01_110,ϵ02_101,ϵ12_011)

        # Mimick symplectic updates by updating first p, and then q
        p0_100 += dt*dp0_100 
        p1_010 += dt*dp1_010 
        p2_001 += dt*dp2_001 

        q0_100 += dt*p0_100*iρ_100 
        q1_010 += dt*p1_010*iρ_010
        q2_001 += dt*p2_001*iρ_001

        return (q0_100,q1_010,q2_001),(p0_100,p1_010,p2_001)

For completeness, we also implement the one dimensional case.

In [9]:
class Virieux1:
    """The two dimensional Virieux scheme."""
    def __init__(self,ρ,C,stag):
        """
        Inputs : 
        - ρ : density, everywhere positive. 
        - C : Hooke tensor, assumes a VTI structure. Assumes C02=C12=0 and C symmetric.
        - stag : staggered grid difference operators
        """
        self.stag = stag
        ar = stag.avg_right
        # Variable locations on the grid shown after subscript
        iρ = 1/ρ
        self.iρ_1 = ar(iρ,0)
        self.C00_0 = C[0,0]

    def step(self,q,p,dt):
        """
        Inputs : 
        - q,p : position and momentum.
        - dt : time step.
        """
        # Variable locations shown after subscript
        dl,dr = self.stag.diff_left, self.stag.diff_right
        C00_00,iρ_1 = self.C00_0,self.iρ_1

        q0_1, = copy.deepcopy(q)
        p0_1, = copy.deepcopy(p)
        
        # Compute the strain tensor ϵ = (Dq+Dq^T)/2
        ϵ00_0 = dl(q0_1,0) 

        # Compute the stress tensor
        σ00_00 = C00_0*ϵ00_0

        # Stress divergence
        dp0_1 = dr(σ00_0,0)

        # Mimick symplectic updates by updating first p, and then q
        p0_1 += dt*dp0_1 
        q0_1 += dt*p0_1*iρ_1 

        return (q0_1,),(p0_1,)

## 3. The Lebedev scheme

The two dimensional Lebedev scheme consists of two Virieux schemes, weakly coupled through the coefficients $C_{02}$ and $C_{12}$ of the Hooke tensor. There are four coupled Virieux schemes in three dimensions. On the other hand, one dimensional Lebedev scheme is equivalent to the Virieux scheme.

The Lebedev scheme uses a staggered grid, and locates the variables as follows.  Compare with the Virieux schemes which stores only one component at each of these positions. 

- *position and momentum*. All components are stored at $(i+1/2,j)\diff x$ and $(i,j+1/2)\diff x$ in two dimensions. (At $(i+1/2,j,k)\diff x$, $(i,j+1/2,k)\diff x$, $(i,j,k+1/2)\diff x$ and $(i+1/2,j+1/2,k+1/2)\diff x$ in three dimensions.)
- *stress and strain*. All components are stored at $(i,j)\diff x$ and $(i+1/2,j+1/2) \diff x$ in two dimensions. (At $(i,j,k)\diff x$, $(i+1/2,j+1/2,k)\diff x$, $(i+1/2,j,k+1/2)\diff x$ and $(i,j+1/2,k+1/2)\diff x$ in three dimensions.)

In [10]:
def eval_Lebedev(qfun,pfun,X,dx,t,dt):
    """
    Evaluate position and momentum at the given position and time, 
    taking into account spatial and temporal grid shifts
    """
    t2=t+dt/2
    vdim=len(X)
    if vdim==1:
        _,X_1 = shifted_grids(X,dx)
        return qfun(t2,X_1),pfun(t,X_1)
    if vdim==2:
        _,X_10,X_01,_ = shifted_grids(X,dx)
        q_10 = qfun(t2,X_10)
        q_01 = qfun(t2,X_01)
        p_10 = pfun(t ,X_10)
        p_01 = pfun(t ,X_01)
        return (q_10,q_01),(p_10,p_01) # Geometry is second
    if vdim==3:
        X_000,X_100,X_010,X_110,X_001,X_101,X_011,X_111 = shifted_grids(X,dx)
        q_100 = qfun(t2,X_100)
        q_010 = qfun(t2,X_010)
        q_001 = qfun(t2,X_001)
        q_111 = qfun(t2,X_111)
        p_100 = pfun(t ,X_100)
        p_010 = pfun(t ,X_010)
        p_001 = pfun(t ,X_001)
        p_111 = pfun(t ,X_111)
        return (q_100,q_010,q_001,q_111), (p_100,p_010,p_001,p_111)

In [11]:
class Lebedev2:
    """The two dimensional Lebedev scheme"""
    def __init__(self,ρ,C,stag):
        """
        Inputs : 
        - ρ : density, everywhere positive. 
        - C : Hooke tensor, assumed to be symmetric.
        - stag : staggered grid difference operators
        """
        self.stag = stag
        ar = stag.avg_right
        # Variable location on the grid shown after underscore
        iρ = 1/ρ 
        self.iρ_10 = ar(iρ,0) 
        self.iρ_01 = ar(iρ,1) 
        self.C_00 = C 
        self.C_11 = ar(C,axis=(2+0,2+1)) 

    def step(self,q,p,dt):
        """
        Inputs : 
        - q,p : position and momentum.
        - dt : time step.
        """
        dl,dr = self.stag.diff_left, self.stag.diff_right
        C_00,C_11,iρ_10,iρ_01 = self.C_00,self.C_11,self.iρ_10,self.iρ_01
         
        # Variable location on the grid shown after underscore
        (q0_10,q1_10), (q0_01,q1_01) = copy.deepcopy(q)
        (p0_10,p1_10), (p0_01,p1_01) = copy.deepcopy(p)

        # Compute the strain tensor ϵ = (Dq+Dq^T)/2
        ϵ00_00 = dl(q0_10,0) 
        ϵ11_00 = dl(q1_01,1) 
        ϵ01_00 = dl(q0_01,1)+dl(q1_10,0) 

        ϵ00_11 = dr(q0_01,0)
        ϵ11_11 = dr(q1_10,1)
        ϵ01_11 = dr(q0_10,1) + dr(q1_01,0) # We omit the factor two in ϵ01 in view of Voigt's notation
        
        # Compute the stress tensor
        σ00_00 = C_00[0,0]*ϵ00_00 + C_00[0,1]*ϵ11_00 + C_00[0,2]*ϵ01_00 
        σ11_00 = C_00[1,0]*ϵ00_00 + C_00[1,1]*ϵ11_00 + C_00[1,2]*ϵ01_00 
        σ01_00 = C_00[2,0]*ϵ00_00 + C_00[2,1]*ϵ11_00 + C_00[2,2]*ϵ01_00

        σ00_11 = C_11[0,0]*ϵ00_11 + C_11[0,1]*ϵ11_11 + C_11[0,2]*ϵ01_11 
        σ11_11 = C_11[1,0]*ϵ00_11 + C_11[1,1]*ϵ11_11 + C_11[1,2]*ϵ01_11 
        σ01_11 = C_11[2,0]*ϵ00_11 + C_11[2,1]*ϵ11_11 + C_11[2,2]*ϵ01_11

        # Stress divergence
        dp0_10 = dr(σ00_00,0) + dl(σ01_11,1) 
        dp1_01 = dl(σ01_11,0) + dr(σ11_00,1) 

        dp0_01 = dl(σ00_11,0) + dr(σ01_00,1) 
        dp1_10 = dr(σ01_00,0) + dl(σ11_11,1) 
#        self.tmp = (dp0_10,dp1_01),(σ00_00,σ11_00,σ01_11),(ϵ00_00,ϵ11_00,ϵ01_11)

        # Mimick symplectic updates by updating first p, and then q
        p0_10 += dt*dp0_10 
        p1_10 += dt*dp1_10 
        p0_01 += dt*dp0_01 
        p1_01 += dt*dp1_01 

        q0_10 += dt*p0_10*iρ_10
        q1_10 += dt*p1_10*iρ_10
        q0_01 += dt*p0_01*iρ_01
        q1_01 += dt*p1_01*iρ_01

        return ((q0_10,q1_10), (q0_01,q1_01)), ((p0_10,p1_10), (p0_01,p1_01))

In [12]:
class Lebedev3:
    """The three dimensional Lebedev scheme"""
    def __init__(self,ρ,C,stag):
        self.stag = stag
        ar = stag.avg_right
        # Variable location on the grid shown after underscore
        iρ = 1/ρ 
        self.iρ_100 = ar(iρ,0) 
        self.iρ_010 = ar(iρ,1)
        self.iρ_001 = ar(iρ,2)
        self.iρ_111 = ar(iρ,(0,1,2))
        self.C_000 = C 
        self.C_110 = ar(C,axis=(2+0,2+1)) 
        self.C_101 = ar(C,axis=(2+0,2+2)) 
        self.C_011 = ar(C,axis=(2+1,2+2)) 

    def step(self,q,p,dt):
        q_100,q_010,q_001,q_111 = copy.deepcopy(q) 
        p_100,p_010,p_001,p_111 = copy.deepcopy(p)
        dl,dr = self.stag.diff_left, self.stag.diff_right
        iρ_100,iρ_010,iρ_001,iρ_111, C_000,C_110,C_101,C_011 = self.iρ_100,self.iρ_010,self.iρ_001,self.iρ_111, self.C_000,self.C_110,self.C_101,self.C_011

        # Compute the strain tensor ϵ = (Dq+Dq^T)/2
        # Voigt convention : 00,11,22, 12,02,01
        ϵ_000 = (dl(q_100[0],0), dl(q_010[1],1), dl(q_001[2],2), dl(q_001[1],2)+dl(q_010[2],1), dl(q_001[0],2)+dl(q_100[2],0), dl(q_010[0],1)+dl(q_100[1],0) )
        ϵ_110 = (dr(q_010[0],0), dr(q_100[1],1), dl(q_111[2],2), dl(q_111[1],2)+dr(q_100[2],1), dl(q_111[0],2)+dr(q_010[2],0), dr(q_100[0],1)+dr(q_010[1],0) )
        ϵ_101 = (dr(q_001[0],0), dl(q_111[1],1), dr(q_100[2],2), dr(q_100[1],2)+dl(q_111[2],1), dr(q_100[0],2)+dr(q_001[2],0), dl(q_111[0],1)+dr(q_001[1],0) )
        ϵ_011 = (dl(q_111[0],0), dr(q_001[1],1), dr(q_010[2],2), dr(q_010[1],2)+dr(q_001[2],1), dr(q_010[0],2)+dl(q_111[2],0), dr(q_001[0],1)+dl(q_111[1],0) )

        # Compute the stress tensor, as a matrix-vector product
        σ_000 = [sum(C_000[i,j]*ϵ_000[j] for j in range(6)) for i in range(6)]
        σ_110 = [sum(C_110[i,j]*ϵ_110[j] for j in range(6)) for i in range(6)]
        σ_101 = [sum(C_101[i,j]*ϵ_101[j] for j in range(6)) for i in range(6)]
        σ_011 = [sum(C_011[i,j]*ϵ_011[j] for j in range(6)) for i in range(6)]

        # Stress divergence  [0,5,4]
        # Voigt convention   [5,1,3]
        #                    [4,3,2]
        dp_100 = (dr(σ_000[0],0) + dl(σ_110[5],1) + dl(σ_101[4],2), 
                  dr(σ_000[5],0) + dl(σ_110[1],1) + dl(σ_101[3],2), 
                  dr(σ_000[4],0) + dl(σ_110[3],1) + dl(σ_101[2],2) )
        
        dp_010 = (dl(σ_110[0],0) + dr(σ_000[5],1) + dl(σ_011[4],2), 
                  dl(σ_110[5],0) + dr(σ_000[1],1) + dl(σ_011[3],2), 
                  dl(σ_110[4],0) + dr(σ_000[3],1) + dl(σ_011[2],2) )
        
        dp_001 = (dl(σ_101[0],0) + dl(σ_011[5],1) + dr(σ_000[4],2), 
                  dl(σ_101[5],0) + dl(σ_011[1],1) + dr(σ_000[3],2), 
                  dl(σ_101[4],0) + dl(σ_011[3],1) + dr(σ_000[2],2) )
        
        dp_111 = (dr(σ_011[0],0) + dr(σ_101[5],1) + dr(σ_110[4],2), 
                  dr(σ_011[5],0) + dr(σ_101[1],1) + dr(σ_110[3],2), 
                  dr(σ_011[4],0) + dr(σ_101[3],1) + dr(σ_110[2],2) )

        self.tmp = (dp_100,dp_010,dp_001,dp_111), (σ_000,σ_110,σ_101,σ_011), (ϵ_000,ϵ_110,ϵ_101,ϵ_011)
        # Mimick symplectic updates by updating first p, and then q
        for i in range(3):
            p_100[i] += dt*dp_100[i]
            p_010[i] += dt*dp_010[i]
            p_001[i] += dt*dp_001[i]
            p_111[i] += dt*dp_111[i]

        for i in range(3):
            q_100[i] += dt*iρ_100*p_100[i]
            q_010[i] += dt*iρ_010*p_010[i]
            q_001[i] += dt*iρ_001*p_001[i]
            q_111[i] += dt*iρ_111*p_111[i]

        return (q_100,q_010,q_001,q_111), (p_100,p_010,p_001,p_111)

## 4. Dispersion relations

**Continuous setting.**
Consider a plane wave 
\begin{align*}
    q(t,x) &= w \be(\<\kw,x\> - \omega t), &
    p(t,x) &= v \be(\<\kw,x\> - \omega t),
\end{align*}
where $\kw$ is the wavenumber, $\omega$ is the pulsation, and $\be(s) := \exp(\ri s)$.

If this wave obeys the elastic wave equation, with constant coefficients, then one easily checks that 
\begin{align*}
    -\ri\omega\, v &= C[\ri\kw] w, &
    -\ri\omega\, w &= v/\rho,
\end{align*}
where $C[\kw]_{il} := \sum_{jk} C_{ijkl} \kw_i \kw_l$.

Note that this relation implies that $w$ is an eigenvector of $C[\kw]$, more precisely $\rho \omega^2 w = C[\ri\kw] w$.

**Discrete setting.**
We first define two helper functions, which correspond to fourier transforms of first order finite difference operators:
\begin{align*}
    \rI_2^h(s) &
    := \frac{\exp(\ri s h/2) - \exp(\ri s h/2)} {\ri h} 
    = \frac{\sin(s h/2)}{h/2} 
    = s - \frac{ s^3 h^2} {24} +  \cO(h^4), \\
    \rI_4^h(s) &
    := \frac 9 8 \frac{\exp(\ri s h/2) - \exp(\ri s h/2)} {\ri h} -\frac 1 {24} \frac{\exp(3\ri s h/2) - \exp(3\ri s h/2)} {\ri h} 
    = \frac 9 8 \frac{\sin(s h/2)}{h/2}  - \frac 1 {24} \frac{\sin(3 s h/2)}{h/2}
    = s - \frac {3 s^5 h^4}{640} + \cO(h^6).
\end{align*}
These expression are second order and fourth order approximations of the identity, respectively. The original ``staggered'' one dimensional finite difference operators are 
\begin{align*}
    \frac{u(x+h/2)-u(x-h/2)} h &= u'(x) + \cO(h^2), \\
    \frac 9 8 \frac{u(x+h/2)-u(x-h/2)} h - \frac 1 8 \frac{u(x+3h/2)-u(x-3h/2)} {3h} &= u'(x) + \cO(h^4).
\end{align*}


In view of the finite difference expressions, and of the locations and times where $q$ and $p$ are evaluated, one obtains the discrete dispersion relation
\begin{align*}
    -\ri \rI_2^{\diff t} (\omega)\, v &= C[\ri \rI_{ox}^{\diff x}\kw] w, &
    -\ri \rI_2^{\diff t}(\omega)\, w &= v/\rho,
\end{align*}
where $ox$ denotes the order of the scheme w.r.t the space variable. This relation implies $\rho \rI_2^{\diff t}(\omega)^2 w = C[\rI_{ox}^{\diff x}\kw] w$.
Given a wave number $\kw$, one can diagonalize the symmetric matrix $C[\rI_{ox}^{\diff x}\kw]$, and thus find the amplitude and pulsations of the corresponding propagation modes.

In [13]:
def _dispersion_I(s,h,order=2):
    """
    Approximation of identity, corresponding to the Fourier transform 
    of first order finite difference operators on staggered grids.
    """
    h2 = h/2
    if order==2: return np.sin(s*h2)/h2
    if order==4: return (9/8)*np.sin(s*h2)/h2-(1/24)*np.sin(3*s*h2)/h2
    raise ValueError("Unsupported order")

def _dispersion_Iinv(s,h):
    """Inverse of I(s,h,order=2)"""
    h2=h/2
    return np.arcsin(s*h2)/h2

def dispersion_Virieux(k,ρ,C,dx,dt,order_x=2):
    """
    Dispersion relation for the Virieux and Lebedev schemes (but Virieux expects C02=C12=0)
    returns the wavemodes and pulsations corresponding to the wavenumber k.
    """
    Ik = _dispersion_I(k,dx,order_x)
    Ck = Hooke(C).contract(Ik)
    if np.ndim(k)==1: Iω2,vq = np.linalg.eigh(Ck/ρ)
    else: Iω2,vq = np.linalg.eigh(np.moveaxis(Ck,(0,1),(-2,-1))/ρ); Iω2 = np.moveaxis(Iω2,-1,0); vq = np.moveaxis(vq,(-2,-1),(0,1)) 
    
    Iω = np.sqrt(Iω2)
    ω = _dispersion_Iinv(Iω,dt)
    vp = -ρ*Iω*vq # Omitting multiplication by i
    return ω,vq,vp

def mk_planewave_e(k,ω,vq,vp):
    """Make an elastic planewave with the specified parameters."""
    def brdcst(v,x): return fd.as_field(v,x[0].shape)
    def expi(t,x): return np.exp(1j*(lp.dot_VV(brdcst(k,x),x) - ω*t)) 
    def q_exact(t,x): return    expi(t,x) * brdcst(vq,x)
    def p_exact(t,x): return 1j*expi(t,x) * brdcst(vp,x)
    return q_exact,p_exact

Let us cross-check our implementations and these dispersion relations, beginning with two-dimensional schemes.

In [14]:
C_VTI2 = Hooke.mica[0].extract_xz().hooke
C_TTI2 = Hooke.mica[0].extract_xz().rotate_by(π/6).hooke
C_VTI3 = Hooke.mica[0].hooke
C_TTI3 = Hooke.mica[0].rotate_by(π/6,(1,2,3)).hooke
for C in (C_VTI2,C_TTI2,C_VTI3,C_TTI3): C/=np.max(C)

for ρ, C_VTI, C, k, order_x, Nx, dt, mode in [
    (1.4, C_VTI2, C_TTI2, [ 2, 3],    2, 10, 0.01, 0),
    (0.9, C_VTI2, C_TTI2, [-1, 1],    4, 10, 0.01, 0),
    (1.6, C_VTI3, C_TTI3, [ 2, 1, 3], 2, 10, 0.01, 0),
    (1.9, C_VTI3, C_TTI3, [-1, 1, 2], 4, 10, 0.01, 0),
]:
    vdim = len(k)
    k = 2*π*np.array(k)
    X, dx = make_domain(Nx,vdim) # The grid points : offset (0,0)
    stag = staggered(dx,order_x,'Periodic')

    # Time discretization
    Nt = 1
    T = Nt*dt

    # Dispersion relation and exact solution
    ω,vq,vp = dispersion_Virieux(k,ρ,C_VTI,dx,dt,order_x)
    q_exact,p_exact = mk_planewave_e(k,ω[mode],vq[:,mode],vp[:,mode])

    q0,p0 = eval_Virieux(q_exact,p_exact,X,dx,0,dt)
    qf,pf = eval_Virieux(q_exact,p_exact,X,dx,T,dt)

    def af(q): return fd.as_field(q,shape=X[0].shape)
    Virieux = (None,Virieux1,Virieux2,Virieux3)[vdim] # Select adequate scheme
    scheme = Virieux(af(ρ),af(C_VTI),stag)
    q1,p1 = scheme.step(q0,p0,dt)
    assert np.allclose(qf,q1)
    assert np.allclose(pf,p1)

    # Now testing the Lebedev scheme
    if vdim==1: continue
    Q0,P0 = eval_Lebedev(q_exact,p_exact,X,dx,0,dt)
    Qf,Pf = eval_Lebedev(q_exact,p_exact,X,dx,T,dt)
    Lebedev = (None,None,Lebedev2,Lebedev3)[vdim]
    Scheme = Lebedev(af(ρ),af(C_VTI),stag)
    Q1,P1 = Scheme.step(Q0,P0,dt)
    assert np.allclose(Qf,Q1)
    assert np.allclose(Pf,P1)

    # For a VTI model, the Lebedev scheme incorporates several decoupled Virieux schemes
    for i in range(vdim): assert np.allclose(Qf[i][i],qf[i])

    # However, the Lebedev scheme also handles general elasticity
    ω,vq,vp = dispersion_Virieux(k,ρ,C,dx,dt,order_x)
    q_exact,p_exact = mk_planewave_e(k,ω[mode],vq[:,mode],vp[:,mode])
    Q0,P0 = eval_Lebedev(q_exact,p_exact,X,dx,0,dt)
    Qf,Pf = eval_Lebedev(q_exact,p_exact,X,dx,T,dt)
    Scheme = Lebedev(af(ρ),af(C),stag)
    Q1,P1 = Scheme.step(Q0,P0,dt)
    assert np.allclose(Qf,Q1)
    assert np.allclose(Pf,P1)


(Old test code commented.)

<!---
# Virieux
ω=ω[mode]; vq=vq[:,mode]; vp=vp[:,mode]
(dp0_10,dp1_01),(σ00_00,σ11_00,σ01_11),(ϵ00_00,ϵ11_00,ϵ01_11) = scheme.tmp

def expi(x): return np.exp(1j*(k[0]*x[0]+k[1]*x[1]-ω*dt/2))
# This works for any vq, not necessarily an eigenvector
assert np.allclose(q0[0]/expi(X_10),vq[0])
assert np.allclose(ϵ00_00/expi(X_00),vq[0]*dspi(0))
assert np.allclose(ϵ11_00/expi(X_00),vq[1]*dspi(1))
assert np.allclose(ϵ01_11/expi(X_11),vq[0]*dspi(1)+vq[1]*dspi(0))
assert np.allclose(σ00_00/expi(X_00),C[0,0]*vq[0]*dspi(0) + C[0,1]*vq[1]*dspi(1))
assert np.allclose(σ11_00/expi(X_00),C[1,0]*vq[0]*dspi(0) + C[1,1]*vq[1]*dspi(1))
assert np.allclose(σ01_11/expi(X_11),C[2,2]*(vq[0]*dspi(1)+vq[1]*dspi(0)))
assert np.allclose(dp0_10/expi(X_10),(C[0,0]*vq[0]*dspi(0)+C[0,1]*vq[1]*dspi(1))*dspi(0) + C[2,2]*(vq[0]*dspi(1)+vq[1]*dspi(0))*dspi(1))
assert np.allclose(dp1_01/expi(X_01),C[2,2]*(vq[0]*dspi(1)+vq[1]*dspi(0))*dspi(0)+(C[1,0]*vq[0]*dspi(0) + C[1,1]*vq[1]*dspi(1))*dspi(1))
Ik = _dispersion_I(k,dx,order_x)
Iω = _dispersion_I(ω,dt)
Ck = Hooke(C).contract(Ik)
assert np.allclose(dp0_10/expi(X_10),-(Ck@vq)[0])
assert np.allclose(dp1_01/expi(X_01),-(Ck@vq)[1])
# Below, need chosen vq
assert np.allclose(Iω*vq,-vp/ρ)
assert np.allclose(Iω*vp,-Ck@vq)
assert np.allclose(p1,pf)

#Iω*vp,Ck@vq, ρ*Iω*vq,vp
#assert np.allclose(_dispersion_I(ω,dt)*vp,Ck@vq)
#Ck@vq,(C[0,0]*vq[0]*dspi(0) + C[0,1]*vq[1]*dspi(1))*dspi(0) + C[2,2]*(vq[0]*dspi(1)+vq[1]*dspi(0))*dspi(1),C[2,2]*(vq[0]*dspi(1)+vq[1]*dspi(0))*dspi(0)+(C[1,0]*vq[0]*dspi(0) + C[1,1]*vq[1]*dspi(1))*dspi(1)

#np.array(q1)-np.array(qf)
def expi(x): return np.exp(1j*(k[0]*x[0]+k[1]*x[1]-ω*dt))
assert np.allclose((qf[0]-q0[0])/expi(X_10),-1j*vq[0]*Iω*dt)
assert np.allclose(pf[0]/expi(X_10),1j*vp[0])
assert np.allclose(qf[0]-q0[0], pf[0]*dt/ρ)
assert np.allclose(q1[0]-q0[0], pf[0]*dt/ρ)

# Lebedev
assert np.allclose(Q0[0][0],q0[0])
assert np.allclose(Q0[1][1],q0[1])
assert np.allclose(P0[0][0],p0[0])
assert np.allclose(P0[1][1],p0[1])

assert np.allclose(Qf[0][0],qf[0])
assert np.allclose(Q1[0][0],q1[0])
assert np.allclose(Qf[0][0],Q1[0][0])
--->

<!---
#C = Hooke.from_Lame(0,1,vdim=2).hooke; ρ=1
C = Hooke.mica[0].extract_xz().hooke; C/=np.max(C); ρ=1.5
k = 2*π*np.array([2,3])
order_x = 2

# The domain is the periodic unit square
Nx = 10
bc = 'Periodic'
X, dx = make_domain(Nx,2) # The grid points : offset (0,0)
stag = staggered(dx,order_x,bc)

# Check that _dispersion_I is indeed the Fourier transform of the finite differences operator
def expi(x): return np.exp(1j*(k[0]*x[0]+k[1]*x[1]))
def dspi(axis): return 1j*_dispersion_I(k[axis],dx,order_x)
X_00,X_10,X_01,X_11 = shifted_grids(X,dx)
assert np.allclose(stag.diff_left( expi(X_10),0),     expi(X)*dspi(0))
assert np.allclose(stag.diff_right(expi(2*X-X_10),0), expi(X)*dspi(0))
assert np.allclose(stag.diff_left( expi(X_01),1),     expi(X)*dspi(1))
assert np.allclose(stag.diff_right(expi(2*X-X_01),1), expi(X)*dspi(1))

# Time step
dt = 0.01
Nt = 1
T = Nt*dt

# Dispersion relation and exact solution
ω,vq,vp = dispersion_Virieux(k,ρ,C,dx,dt)
mode = 0
q_exact,p_exact = mk_planewave_e(k,ω[mode],vq[:,mode],vp[:,mode])

q0,p0 = eval_Virieux(q_exact,p_exact,X,dx,0,dt)
qf,pf = eval_Virieux(q_exact,p_exact,X,dx,T,dt)

def af(q): return fd.as_field(q,shape=X[0].shape)
scheme = Virieux2(af(ρ),af(C),stag)
q1,p1 = scheme.step(q0,p0,dt)
assert np.allclose(qf,q1)
assert np.allclose(pf,p1)

# Now testing the Lebedev scheme
Q0,P0 = eval_Lebedev(q_exact,p_exact,X,dx,0,dt)
Qf,Pf = eval_Lebedev(q_exact,p_exact,X,dx,T,dt)
Scheme = Lebedev2(af(ρ),af(C),stag)
Q1,P1 = Scheme.step(Q0,P0,dt)
assert np.allclose(Qf,Q1)
assert np.allclose(Pf,P1)

# For a VTI model, the Lebedev scheme consists of two decoupled Virieux schemes
assert np.allclose(Qf[0][0],qf[0])
assert np.allclose(Qf[1][1],qf[1])

# However, the Lebedev scheme also handles general elasticity
C = Hooke.mica[0].extract_xz().rotate_by(π/6).hooke; C/=np.max(C); ρ=1.5
ω,vq,vp = dispersion_Virieux(k,ρ,C,dx,dt)
mode = 1
q_exact,p_exact = mk_planewave_e(k,ω[mode],vq[:,mode],vp[:,mode])
Q0,P0 = eval_Lebedev(q_exact,p_exact,X,dx,0,dt)
Qf,Pf = eval_Lebedev(q_exact,p_exact,X,dx,T,dt)
Scheme = Lebedev2(af(ρ),af(C),stag)
Q1,P1 = Scheme.step(Q0,P0,dt)
assert np.allclose(Qf,Q1)
assert np.allclose(Pf,P1)
--->

<!---
C = Hooke.mica[0].hooke; C/=np.max(C); ρ=1.5
k = 2*π*np.array([2,-1,3])
order_x = 2

# The domain is the periodic unit square
Nx = 10
bc = 'Periodic'
X, dx = make_domain(10,3) # The grid points : offset (0,0)
stag = staggered(dx,order_x,bc)

# Check that _dispersion_I is indeed the Fourier transform of the finite differences operator
#def expi(x): return np.exp(1j*(k[0]*x[0]+k[1]*x[1]))
#def dspi(axis): return 1j*_dispersion_I(k[axis],dx,order_x)
#X_00,X_10,X_01,X_11 = shifted_grids(X,dx)
#assert np.allclose(stag.diff_left( expi(X_10),0),     expi(X)*dspi(0))
#assert np.allclose(stag.diff_right(expi(2*X-X_10),0), expi(X)*dspi(0))
#assert np.allclose(stag.diff_left( expi(X_01),1),     expi(X)*dspi(1))
#assert np.allclose(stag.diff_right(expi(2*X-X_01),1), expi(X)*dspi(1))

# Time step
dt = 0.01
Nt = 1
T = Nt*dt

# Dispersion relation and exact solution
ω,vq,vp = dispersion_Virieux(k,ρ,C,dx,dt)
mode = 0
q_exact,p_exact = mk_planewave_e(k,ω[mode],vq[:,mode],vp[:,mode])

q0,p0 = eval_Virieux(q_exact,p_exact,X,dx,0,dt)
qf,pf = eval_Virieux(q_exact,p_exact,X,dx,T,dt)

def af(q): return fd.as_field(q,shape=X[0].shape)
scheme = Virieux3(af(ρ),af(C),stag)
q1,p1 = scheme.step(q0,p0,dt)
assert np.allclose(qf,q1)
assert np.allclose(pf,p1)

# Now testing the Lebedev scheme
Q0,P0 = eval_Lebedev(q_exact,p_exact,X,dx,0,dt)
Qf,Pf = eval_Lebedev(q_exact,p_exact,X,dx,T,dt)
assert np.allclose(Qf[0][0],qf[0])
assert np.allclose(Qf[1][1],qf[1])
assert np.allclose(Qf[2][2],qf[2])
assert np.allclose(Pf[0][0],pf[0])
assert np.allclose(Pf[1][1],pf[1])
assert np.allclose(Pf[2][2],pf[2])

Scheme = Lebedev3(af(ρ),af(C),stag)
Q1,P1 = Scheme.step(Q0,P0,dt)
# For a VTI model, the Lebedev scheme consists of 2^(d-1) decoupled Virieux schemes
(dp0_100,dp1_010,dp2_001),(σ00_000,σ11_000,σ22_000,σ01_110,σ02_101,σ12_011),(ϵ00_000,ϵ11_000,ϵ22_000,ϵ01_110,ϵ02_101,ϵ12_011) = scheme.tmp
(dp_100,dp_010,dp_001,dp_111), (σ_000,σ_110,σ_101,σ_011), (ϵ_000,ϵ_110,ϵ_101,ϵ_011) = Scheme.tmp

assert np.allclose(ϵ00_000,ϵ_000[0])
assert np.allclose(dp0_100,dp_100[0])
assert np.allclose(dp1_010,dp_010[1])
assert np.allclose(dp2_001,dp_001[2])
assert np.allclose(P0[0][0],p0[0])
assert np.allclose(Pf[0][0],pf[0])
assert np.allclose(Qf[0][0],Q1[0][0])

assert np.allclose(Qf,Q1)
assert np.allclose(Pf,P1)


# However, the Lebedev scheme also handles general elasticity
C = Hooke.mica[0].rotate_by(π/6,(1,2,3)).hooke; C/=np.max(C); ρ=1.5
ω,vq,vp = dispersion_Virieux(k,ρ,C,dx,dt)
mode = 1
q_exact,p_exact = mk_planewave_e(k,ω[mode],vq[:,mode],vp[:,mode])
Q0,P0 = eval_Lebedev(q_exact,p_exact,X,dx,0,dt)
Qf,Pf = eval_Lebedev(q_exact,p_exact,X,dx,T,dt)
Scheme = Lebedev3(af(ρ),af(C),stag)
Q1,P1 = Scheme.step(Q0,P0,dt)
assert np.allclose(Qf,Q1)
assert np.allclose(Pf,P1)
--->

## 5. Isotropic dispersion

Recall that an isotropic Hooke tensor is defined in terms of two Lame parameters $\lambda$ and $\mu$, and satisfies 
$$
    C(m,m) := \sum_{ijkl} C_{ijkl} m_{ij} m_{kl} = \lambda \Tr(m)^2 + 2 \mu \Tr(m^2).
$$
We defined, for any wave number $\kw$, a symmetric matrix $C[\kw]$ as follows
$$
    C[\kw]_{il} := \sum_{jk} \kw_{j}\kw_k,
$$
Noting that $v^\top C[\kw] v = C(m,m)$ with $2 m = \kw v^\top + v \kw^\top$, we obtain the explicit expression is the isotropic case
$$
    C[\kw] = \lambda \kw \kw^\top + \mu(|\kw|^2 \Id + \kw \kw^\top),
$$
which has eigenvalues $\mu |\kw|^2$ and $(\lambda+2\mu)|\kw|^2$, with eigenspaces $\kw^\perp$ and $\kw\bR$. 
Using the dispersion relation $\rho \omega^2 v = C[k] v$, we obtain the associated wave speeds
\begin{align*}
    V_S &= \sqrt{\frac \mu \rho},& V_P =& \sqrt{\frac{\lambda+2 \mu} \rho}.
\end{align*}

A measure of quality of an elastic wave scheme is how well it reproduces these wave speeds, in other words the (lack of) dispersion relation in the isotropic setting.

**Virieux scheme.**
The modified dispersion relation reads
$$
    \rho \rI_2^{\diff t}(\omega) v = C[\rI_{ox}^{\diff x}(\kw)] v
$$

**Selling based scheme**
The effect is more complex, and depends on the sign of $\lambda$, which leads to different Selling decompositions. 
We begin with the two dimensional and second order case, and do not show all intermediate computations. The other cases are treated similarly.
Define 
$$
    E_2^{\diff x}(v) := 
    \begin{pmatrix}
    \rI_2^{\diff x}(v_0)^2 & \rI_2^{2\diff x}(v_0) \rI_2^{2\diff x}(v_1) \\
    \rI_2^{2\diff x}(v_0) \rI_2^{2\diff x}(v_1) & \rI_2^{\diff x}(v_1)^2 
    \end{pmatrix},
$$
which is an approximation of $v v^\top$. Note the step $2 \diff x$ for the off-diagonal coefficients, which are thus less faithful than the diagonal coefficients. The numerical replacement for the dispersion matrix $C[k]$ is obtained as follows : 

- for $\lambda \geq 0$
$$
    2\mu \begin{pmatrix}\rI_2^{\diff x}(k_0)^2 & \\ & \rI_2^{\diff x}(k_1)^2 \end{pmatrix} + \mu E^{\diff x}_2( k_1,k_0) + \lambda E^{\diff x}_2(k)
$$
- for $\lambda\leq 0$ (note that $\lambda \geq -\mu$ by assumption)
$$
2(\mu+\lambda) \begin{pmatrix}\rI_2^{\diff x}(k_0)^2 & \\ & \rI_2^{\diff x}(k_1)^2 \end{pmatrix} + \mu E^{\diff x}_2( k_1,k_0) - \lambda E^{\diff x}_2(-k_0,k_1)
$$


In other words the diagonal coefficients are the same as for the Virieux scheme, but the off-diagonal coefficients are slightly less accurate, corresponding to a twice larger grid step. (The situation is a little bit more complex with anisotropic Hooke tensors.) 

There appears to be a price to handling general Hooke tensors, which is either a multiplication of the unknowns in the Lebedev case, or slightly degraded off-diagonal coefficients in the dispersion relation in the proposed symplectic scheme. Attempts to design a symplectic staggered grid scheme for general anisotropy and with the same dispersion as Virieux's in the isotropic case were so far infructuous. 
However, the numerical dispersion of the proposed scheme is favorable w.r.t the Lebedev if we adjust the number of points per wavelength so as to take into account the additional Lebedev unknowns. 

In the next cell, we compute the numerical pulsation in terms of a particular wavenumber, for the different numerical schemes.

In [15]:
# Model parameters
λ = -0.4 # 1
μ = 1.
ρ = 1.5
k = np.array((0.5,1))

# Exact speeds
Vs = np.sqrt(μ/ρ); Vp = np.sqrt((λ+2*μ)/ρ)
k_norm = np.linalg.norm(k)
ω_exact = np.array([Vs,Vp])*k_norm

# Discretization parameters
dx = 0.8
dt = dx/1.3
order_x = 2

# Computation of the numerical speeds
C = Hooke.from_Lame(λ,μ).hooke 
ω_Virieux,_,_ = dispersion_Virieux(k,ρ,C,dx,dt,order_x=order_x)
ω_Lebedev,_,_ = dispersion_Virieux(k,ρ,C,dx*np.sqrt(2),dt,order_x=order_x) # dx*sqrt(2) takes into accout 2x density of unknowns
ω_Selling,_,_ = dispersion_e(k,ρ,C,dx,dt,order_x=order_x)


print(f"{ω_exact=}, points per wavelength : {2*π/(k_norm*dx)=}, CFL : {dt/(Vs*dx)=}")
print(f"{ω_Virieux-ω_exact=}")
print(f"{ω_Lebedev-ω_exact=}")
print(f"{ω_Selling-ω_exact=}")

ω_exact=array([0.91287093, 1.15470054]), points per wavelength : 2*π/(k_norm*dx)=7.024814731040726, CFL : dt/(Vs*dx)=0.9421114395319916
ω_Virieux-ω_exact=array([-0.00889369, -0.00191558])
ω_Lebedev-ω_exact=array([-0.02981232, -0.02903098])
ω_Selling-ω_exact=array([-0.00021163, -0.00900189])


However, the dispersion relation features some axis dependent bias, hence values obtained for a specific wavenumber may not be representative. In the next cell, we compute the relation for all possible directions.

**Choice of level set.**
Consider a wave propagating at a speed $V = \omega/\|k\|$. If $\omega=V$ (adimensionized units), then $\|k\|=1$, and therefore the number of points per wavelength is $2\pi/(k\diff x) = 2 \pi/\diff x$  (and the CFL is $\diff t/(V \diff x)$).

In [16]:
# Try changing the following parameters
order_x=2
dx = 1.2
dt = dx/1.8
λ = 0.4 # Try any value >=-1

# No need to change
μ = 1
ρ = 1
C = Hooke.from_Lame(λ,μ).hooke 
Vs = np.sqrt(μ/ρ); Vp = np.sqrt((λ+2*μ)/ρ)

print(f"Points per wavelength {2*π/dx=:.3f}, {order_x=}, {λ/μ=}")
print(f"Blue : Lebedev (worst). Green : Virieux (best, but VTI only). Red : Selling.")

ak = np.linspace(-1.2,1.2)
ks = np.array(np.meshgrid(ak,ak,indexing='ij'))
def af(x): return fd.as_field(x,ks[0].shape)

ω_Virieux,_,_  = dispersion_Virieux(ks,ρ,C,dx,dt,order_x=order_x)
# The next line adjusts dx to take into account the points per wavelength w.r.t Lebedev 
# Multiply dx by 2**(1/2) for 2D, and 2**(2/3) for 3D
ω_Lebedev,_,_  = dispersion_Virieux(ks,ρ,C,dx*np.sqrt(2),dt,order_x=order_x)
ω_Selling,_,_  = dispersion_e(ks,af(ρ),af(C),dx,dt,order_x=order_x,order_t=2)


fig,ax = plt.subplots(1,2,figsize=(12,6))
ax[0].set_title("Shear wave dispersions"); ax[0].axis('equal')
ax[0].contour(*ks,ω_Lebedev[0],levels=[Vs],colors='blue')
ax[0].contour(*ks,ω_Virieux[0],levels=[Vs],colors='green')
ax[0].contour(*ks,ω_Selling[0],levels=[Vs],colors='red')
ax[0].add_patch(plt.Circle((0,0),1,fill=False))

ax[1].set_title("Pressure wave dispersions"); ax[1].axis('equal')
ax[1].contour(*ks,ω_Lebedev[1],levels=[Vp],colors='blue')
ax[1].contour(*ks,ω_Virieux[1],levels=[Vp],colors='green')
ax[1].contour(*ks,ω_Selling[1],levels=[Vp],colors='red')
ax[1].add_patch(plt.Circle((0,0),1,fill=False));

Points per wavelength 2*π/dx=5.236, order_x=2, λ/μ=0.4
Blue : Lebedev (worst). Green : Virieux (best, but VTI only). Red : Selling.


In the VTI case, for the mica medium, the Virieux and proposed dispersions are almost identical, whereas the Lebedev scheme is again less accurate.

In [17]:
# Try changing the following parameters
order_x=2
dx = 1.2
dt = dx/1.8
C = Hooke.mica[0].extract_xz().hooke

# No need to change
ρ = 1

print(f"Points per wavelength {2*π/dx=:.3f}, {order_x=}, {λ/μ=}")
print(f"Blue : Lebedev (worst). Green : Virieux (best, but VTI only). Red : Selling.")

ak = np.linspace(-1.2,1.2)
ks = np.array(np.meshgrid(ak,ak,indexing='ij'))
def af(x): return fd.as_field(x,ks[0].shape)


ϵ=1e-6; ω_exact,_,_ = dispersion_Virieux(ks,ρ,C,ϵ*dx,ϵ*dt,order_x=order_x) # Too lazy
ω_Virieux,_,_  = dispersion_Virieux(ks,ρ,C,dx,dt,order_x=order_x)
ω_Lebedev,_,_  = dispersion_Virieux(ks,ρ,C,dx*np.sqrt(2),dt,order_x=order_x)
ω_Selling,_,_  = dispersion_e(ks,af(ρ),af(C),dx,dt,order_x=order_x,order_t=2)


fig,ax = plt.subplots(1,2,figsize=(12,6))
ax[0].set_title("Shear wave dispersions"); ax[0].axis('equal')
ω0 = 0.7*ω_exact[0].mean()
ax[0].contour(*ks,ω_Lebedev[0],levels=[ω0],colors='blue')
ax[0].contour(*ks,ω_Virieux[0],levels=[ω0],colors='green')
ax[0].contour(*ks,ω_Selling[0],levels=[ω0],colors='red')
ax[0].contour(*ks,ω_exact[0],levels=[ω0],colors='black')

ax[1].set_title("Pressure wave dispersions"); ax[1].axis('equal')
ω1 = 0.8*ω_exact[1].mean()
ax[1].contour(*ks,ω_Lebedev[1],levels=[ω1],colors='blue')
ax[1].contour(*ks,ω_Virieux[1],levels=[ω1],colors='green')
ax[1].contour(*ks,ω_Selling[1],levels=[ω1],colors='red')
ax[1].contour(*ks,ω_exact[1],levels=[ω1],colors='black');

Points per wavelength 2*π/dx=5.236, order_x=2, λ/μ=0.4
Blue : Lebedev (worst). Green : Virieux (best, but VTI only). Red : Selling.


Finally, we consider the TTI case. Note that the Virieux scheme does not apply, but that the `dispersion_Virieux` function does, with the correct result for Lebedev as illustrated above. In this setting find that :
- the Lebedev scheme is more accurate for the shear wave.
- the proposed symplectic scheme is more accurate for the pressure wave.

In [18]:
# Try changing the following parameters
order_x=4
dx = 1.5
dt = dx/1.8
C = Hooke.mica[0].extract_xz().rotate_by(π/8).hooke

# No need to change
ρ = 1

print(f"Points per wavelength {2*π/dx=:.3f}, {order_x=}, {λ/μ=}")
print(f"Blue : Lebedev. Red : Selling.")

ak = np.linspace(-1.2,1.2)
ks = np.array(np.meshgrid(ak,ak,indexing='ij'))
def af(x): return fd.as_field(x,ks[0].shape)


ϵ=1e-6; ω_exact,_,_ = dispersion_Virieux(ks,ρ,C,ϵ*dx,ϵ*dt,order_x=order_x) # Too lazy
ω_Lebedev,_,_  = dispersion_Virieux(ks,ρ,C,dx*np.sqrt(2),dt,order_x=order_x)
ω_Selling,_,_  = dispersion_e(ks,af(ρ),af(C),dx,dt,order_x=order_x,order_t=2)


fig,ax = plt.subplots(1,2,figsize=(12,6))
ax[0].set_title("Shear wave dispersions"); ax[0].axis('equal')
ω0 = 0.7*ω_exact[0].mean()
ax[0].contour(*ks,ω_Lebedev[0],levels=[ω0],colors='blue')
ax[0].contour(*ks,ω_Selling[0],levels=[ω0],colors='red')
ax[0].contour(*ks,ω_exact[0],levels=[ω0],colors='black')

ax[1].set_title("Pressure wave dispersions"); ax[1].axis('equal')
ω1 = 0.8*ω_exact[1].mean()
ax[1].contour(*ks,ω_Lebedev[1],levels=[ω1],colors='blue')
ax[1].contour(*ks,ω_Selling[1],levels=[ω1],colors='red')
ax[1].contour(*ks,ω_exact[1],levels=[ω1],colors='black');

Points per wavelength 2*π/dx=4.189, order_x=4, λ/μ=0.4
Blue : Lebedev. Red : Selling.


## 6. Energy conservation

The proposed symplectic scheme, based on Selling's decomposition, conserves a perturbed energy. As a result, the true system energy is conserved up to a multiplicative constant (which is close to one and if the CFL is satisfied strictly).

In contrast, the energy of the Lebedev scheme offers no such guarantee. In the following (admittedly unphysical) example where the Hooke tensors are chosen at random, the we observe that : 

- The Lebedev energy oscillates widly. (However, it does not seem to tend to $0$ or $\infty$, fortunately.)
- For non-VTI hooke tensors, the Lebedev energy usually overestimates the true energy. In contrast it appears to oscillate centered on the true energy in the VTI case.
- Oscillations are larger for the 4th order variant of the Lebedev scheme.

In [19]:
Nt = 2000
Nx = 10
vdim = 2
order_x = 2
bc = 'Periodic'
cfl = 0.7
VTI = False

np.random.seed(42)
X, dx = make_domain(Nx,vdim) # The grid points : offset (0,0)
stag = staggered(dx,order_x,bc)

C = MakeRandomTensor(3,(Nx,Nx),0.5); C/=np.max(C,axis=(0,1)) # Random Hooke elasticity tensors
ρ = np.random.rand(Nx,Nx)+0.5

if VTI: C[0,2]=C[1,2]=C[2,0]=C[2,1]=0

q0_Lebedev = np.random.rand(2,2,Nx,Nx)
p0_Lebedev = np.random.rand(2,2,Nx,Nx)

q0_Selling = np.array((q0_Lebedev[0,0],q0_Lebedev[1,1]))
p0_Selling = np.array((p0_Lebedev[0,0],p0_Lebedev[1,1]))

q_Lebedev,p_Lebedev,q_Selling,p_Selling = copy.deepcopy((q0_Lebedev,p0_Lebedev,q0_Selling,p0_Selling))

scheme_Lebedev = Lebedev2(ρ,C,stag).step
scheme_Virieux = Virieux2(ρ,C,stag).step
WaveH = AnisotropicWave.ElasticHamiltonian_Sparse(ρ[None,None],C,dx,order_x,bc=bc)
scheme_Selling = WaveH.Verlet_p

energy_Selling  = lambda q,p : WaveH.H(q,p)
energy_Selling2 = lambda q,p : WaveH.H_p(q,p,dt,order=2)
energy_Lebedev  = lambda q,p : 0.5*(WaveH.H(np.array((q[0][0],q[1][1])),np.array((p[0][0],p[1][1]))) \
                                  + WaveH.H(np.array((q[0][1],q[1][0])),np.array((p[0][1],p[1][0]))))

hist_Selling  = []
hist_Selling2 = []
hist_Lebedev  = []

dt = cfl*WaveH.dt_max()
for i in range(Nt):
    q_Lebedev,p_Lebedev = scheme_Lebedev(q_Lebedev,p_Lebedev,dt)    
    q_Selling,p_Selling = scheme_Selling(q_Selling,p_Selling,dt)

    hist_Lebedev.append(energy_Lebedev(q_Lebedev,p_Lebedev))
    hist_Selling.append(energy_Selling(q_Selling,p_Selling))
    hist_Selling2.append(energy_Selling2(q_Selling,p_Selling))

In [20]:
plt.title(f"Numerical energy (non-)conservation. {order_x=}, {VTI=}")
plt.plot(hist_Selling,label="Selling energy")
plt.plot(hist_Selling2,label="Selling invariant energy")
plt.plot(hist_Lebedev,label="Lebedev energy")
plt.legend();

## 7. Grid decoupling in the Lebedev scheme

Let us verify that the Lebedev scheme features two independent subgrids in the case of VTI tensors, each of them equivalent to a Virieux scheme.

In [21]:
# Choose the problem parameters
np.random.seed(42)
q0_Lebedev = np.random.rand(2,2,Nx,Nx)
p0_Lebedev = np.random.rand(2,2,Nx,Nx)

def af(q): return fd.as_field(q,shape=X[0].shape)
#ρ = af(1)
ρ = np.random.rand(Nx,Nx)+0.5

# We need C to have a VTI structure
#C = af(Hooke.from_Lame(1,1).hooke)
C = MakeRandomTensor(3,(Nx,Nx),0.5); C/=np.max(C,axis=(0,1)); C[0,2]=C[1,2]=C[2,0]=C[2,1]=0 # Random VTI elasticity tensors

# The Virieux unknowns are extracted from the Lebedev unknowns in a specific staggered manner
def extract_a(q): return (q[0][0],q[1][1])
q0_Virieux_a = extract_a(q0_Lebedev)
p0_Virieux_a = extract_a(p0_Lebedev)
def extract_b(q): return np.roll(q[1][0],-1,axis=0),np.roll(q[0][1],-1,axis=1)
q0_Virieux_b = extract_b(q0_Lebedev)
p0_Virieux_b = extract_b(p0_Lebedev)

scheme_Lebedev = Lebedev2(ρ,C,stag)
scheme_Virieux_a = Virieux2(ρ,C,stag)

# We must carefully shift coefficients of the second Virieux scheme
scheme_Virieux_b = copy.copy(scheme_Virieux_a)
scheme_Virieux_b.iρ_10 = np.roll(scheme_Virieux_a.iρ_01,-1,axis=0)
scheme_Virieux_b.iρ_01 = np.roll(scheme_Virieux_a.iρ_10,-1,axis=1)
scheme_Virieux_b.C00_00 = scheme_Lebedev.C_11[0,0]
scheme_Virieux_b.C01_00 = scheme_Lebedev.C_11[0,1]
scheme_Virieux_b.C11_00 = scheme_Lebedev.C_11[1,1]
scheme_Virieux_b.C22_11 = np.roll(scheme_Lebedev.C_00[2,2],(-1,-1),axis=(0,1))

q1_Lebedev,p1_Lebedev = scheme_Lebedev.step(q0_Lebedev,p0_Lebedev,dt)
q1_Virieux_a,p1_Virieux_a = scheme_Virieux_a.step(q0_Virieux_a,p0_Virieux_a,dt)
q1_Virieux_b,p1_Virieux_b = scheme_Virieux_b.step(q0_Virieux_b,p0_Virieux_b,dt)
assert np.allclose(q1_Virieux_a,extract_a(q1_Lebedev))
assert np.allclose(q1_Virieux_b,extract_b(q1_Lebedev))